In [56]:
import numpy as np
import pandas as pd
import pickle
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from econml.grf import CausalForest
from sklift.metrics import qini_auc_score
from sklift.viz import plot_qini_curve

from matplotlib import pyplot as plt

# Train models

In [57]:
data = pd.read_csv("bsc_project_set.csv", index_col=0)

xs = data.copy()

y = xs["mort_28"]
w = xs["peep_regime"]

w = pd.Series(map(lambda x: 0 if x == "low" else 1, w))
y = pd.Series(map(lambda x: 1 if x == False else 0, y))

# xs = xs.drop(columns=["id", "mort_28", "peep", "peep_regime"])
selected_columns = ["age", "weight", "pf_ratio", "po2", "pco2", "driving_pressure", "fio2", "minute_volume", "plateau_pressure"]
xs = xs[selected_columns]
columns_x = xs.columns

norm_xs = (xs - xs.mean())/xs.std()

imputer = KNNImputer(n_neighbors=2, weights="uniform")
imp_xs = imputer.fit_transform(norm_xs)

imp_xs = pd.DataFrame(data=imp_xs, columns=columns_x)

imp_xs

,age,weight,pf_ratio,po2,pco2,driving_pressure,fio2,minute_volume,plateau_pressure
0,0.732971,0.183841,-0.026374,-0.049713,-1.422636,-0.364306,-0.564826,-0.543738,0.705302
1,-1.005004,1.422974,-0.637005,-0.385550,-0.768253,-1.141620,0.758913,3.032717,-0.620678
2,1.353676,-0.381057,-0.171336,-0.406111,-0.403630,1.468983,-0.434729,-0.035368,0.317210
3,1.353676,0.100018,-0.319055,-1.169822,-0.403630,1.395651,-1.503965,-0.868889,0.640620
4,0.050195,-0.654395,-0.577049,-0.737052,-0.334933,-1.072479,-0.385943,-1.051634,-1.236080
...,...,...,...,...,...,...,...,...,...
3936,0.919183,-1.128181,-0.711144,0.068026,-0.395043,1.219655,0.541000,-0.428608,0.252528
3937,0.732971,0.475402,-0.527518,-0.793421,-0.300585,-0.540301,-0.411497,-0.966353,-1.028174
3938,0.236407,0.438957,-0.701828,-0.554080,0.137359,0.281012,1.377339,-0.299739,1.869576
3939,0.298477,0.245798,-0.425805,-0.814696,-0.156321,0.075683,0.150708,-0.298318,0.113924


In [58]:
full_data = imp_xs.assign(W=w, Y=y)

In [59]:
# # S-learner
# regr_s = RandomForestRegressor(max_depth=5, min_samples_split=5, n_estimators=500, max_samples=0.5)
# regr_s.fit(full_data.drop(columns=["Y"]), full_data["Y"])
#
# # T-learner
# regr0_t = RandomForestRegressor(max_depth=3, min_samples_split=5, n_estimators=500, max_samples=0.5)
# regr1_t = RandomForestRegressor(max_depth=3, min_samples_split=5, n_estimators=500, max_samples=0.5)
# regr0_t.fit(full_data.query("W==0").drop(columns=["W", "Y"]), full_data.query("W==0")["Y"])
# regr1_t.fit(full_data.query("W==1").drop(columns=["W", "Y"]), full_data.query("W==1")["Y"])
#
# # Causal forest
# causal_forest = CausalForest(max_depth=5, min_samples_split=10, n_estimators=2500)
# causal_forest.fit(X=full_data.drop(columns=["W", "Y"]), T=full_data["W"], y=full_data["Y"])

CausalForest(max_depth=5, n_estimators=2500)

In [60]:
# with open("pickled_models/causal_forest_model.pkl", "wb") as f:
#     pickle.dump(causal_forest, f, protocol=5)
#
# with open("pickled_models/s_learner_model.pkl", "wb") as f:
#     pickle.dump(regr_s, f, protocol=5)
#
# with open("pickled_models/t_learner_model_0.pkl", "wb") as f:
#     pickle.dump(regr0_t, f, protocol=5)
#
# with open("pickled_models/t_learner_model_1.pkl", "wb") as f:
#     pickle.dump(regr1_t, f, protocol=5)

In [61]:
with open("pickled_models/causal_forest_model.pkl", "rb") as f:
    causal_forest = pickle.load(f)

with open("pickled_models/s_learner_model.pkl", "rb") as f:
    regr_s = pickle.load(f)

with open("pickled_models/t_learner_model_0.pkl", "rb") as f:
    regr0_t = pickle.load(f)

with open("pickled_models/t_learner_model_1.pkl", "rb") as f:
    regr1_t = pickle.load(f)

In [62]:
causal_forest.predict(X=full_data.drop(columns=["W", "Y"]))

array([[-0.20653261],
       [-0.15808798],
       [-0.2249416 ],
       ...,
       [-0.12809791],
       [-0.1040518 ],
       [-0.14148118]])

In [63]:
regr_s.predict(full_data.drop(columns=["W", "Y"]).assign(**{"W": 1})) - regr_s.predict(full_data.drop(columns=["W", "Y"]).assign(**{"W": 0}))

array([-0.07236983, -0.07370414, -0.08657526, ..., -0.03767396,
       -0.04392505, -0.07302504])

In [64]:
regr1_t.predict(full_data.drop(columns=["W", "Y"])) - regr0_t.predict(full_data.drop(columns=["W", "Y"]))

array([-0.20844512, -0.23685587, -0.29349542, ..., -0.15917994,
       -0.15326956, -0.30980973])

In [65]:
def predict_ite_causal_forest(forest_model, data):
    """
    Predicts ITEs using causal forest
    :param forest_model: the causal forest model, created using pickle.load()
    :param data: pandas dataframe with RCT data, created by for example pandas.read_csv("filename", index_col=0)
    :return: Predicted ITEs
    """

    xs = data.drop(columns=["peep_regime", "mort_28", "id"])
    xs.loc[xs["sex"] == "M", "sex"] = 0
    xs.loc[xs["sex"] == "F", "sex"] = 1
    xs_columns = xs.columns

    norm_xs = (xs - xs.mean())/xs.std()

    imputer = KNNImputer()
    imp_xs = imputer.fit_transform(norm_xs)
    imp_xs = pd.DataFrame(data=imp_xs, columns=xs_columns)

    selected_columns = ["age", "weight", "pf_ratio", "po2", "pco2", "driving_pressure", "fio2", "minute_volume", "plateau_pressure"]
    imp_xs = imp_xs[selected_columns]

    return forest_model.predict(X=imp_xs)

In [68]:
def predict_ite_s_learner(s_learner_model, data):
    """
    Predicts ITEs using S-learner
    :param s_learner_model: the S-learner model, created using pickle.load()
    :param data: pandas dataframe with RCT data, created by for example pandas.read_csv("filename", index_col=0)
    :return: Predicted ITEs
    """

    xs = data.drop(columns=["peep_regime", "mort_28", "id"])
    xs.loc[xs["sex"] == "M", "sex"] = 0
    xs.loc[xs["sex"] == "F", "sex"] = 1
    xs_columns = xs.columns

    norm_xs = (xs - xs.mean())/xs.std()

    imputer = KNNImputer()
    imp_xs = imputer.fit_transform(norm_xs)
    imp_xs = pd.DataFrame(data=imp_xs, columns=xs_columns)

    selected_columns = ["age", "weight", "pf_ratio", "po2", "pco2", "driving_pressure", "fio2", "minute_volume", "plateau_pressure"]
    imp_xs = imp_xs[selected_columns]

    return s_learner_model.predict(imp_xs.assign(**{"W": 1})) - s_learner_model.predict(imp_xs.assign(**{"W": 0}))

In [71]:
def predict_ite_t_learner(t_learner_model_0, t_learner_model_1, data):
    """
    Predicts ITEs using S-learner
    :param t_learner_model_0: the T-learner base model for Y_0, created using pickle.load()
    :param t_learner_model_1: the T-learner base model for Y_1, created using pickle.load()
    :param data: pandas dataframe with RCT data, created by for example pandas.read_csv("filename", index_col=0)
    :return: Predicted ITEs
    """

    xs = data.drop(columns=["peep_regime", "mort_28", "id"])
    xs.loc[xs["sex"] == "M", "sex"] = 0
    xs.loc[xs["sex"] == "F", "sex"] = 1
    xs_columns = xs.columns

    norm_xs = (xs - xs.mean())/xs.std()

    imputer = KNNImputer()
    imp_xs = imputer.fit_transform(norm_xs)
    imp_xs = pd.DataFrame(data=imp_xs, columns=xs_columns)

    selected_columns = ["age", "weight", "pf_ratio", "po2", "pco2", "driving_pressure", "fio2", "minute_volume", "plateau_pressure"]
    imp_xs = imp_xs[selected_columns]

    return t_learner_model_1.predict(imp_xs) - t_learner_model_0.predict(imp_xs)

# Evaluate

In [74]:
s_learner_ite = []
t_learner_ite = []
causal_forest_ite = []
real_w = []
real_y = []

In [76]:
observed_data = pd.DataFrame.from_dict({"W": real_w, "Y": real_y}, orient="columns")

print("ATE:")
print(f"Real: {np.mean(observed_data.query('W==1')['Y']) - np.mean(observed_data.query('W==0')['Y'])}")
print(f"S-learner: {np.mean(s_learner_ite)}")
print(f"T-learner: {np.mean(t_learner_ite)}")
print(f"Causal forest: {np.mean(causal_forest_ite)}")

Real ATE: nan
S-learner predicted ATE: nan
T-learner predicted ATE: nan
Causal forest predicted ATE: nan


D:\Delft\Year 3\Q4\code\research-project\venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Delft\Year 3\Q4\code\research-project\venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
fig, ax = plt.subplots(1, 1)
plot_qini_curve(y_true=real_y, uplift=s_learner_ite, treatment=real_w, perfect=False, random=False, name="S-learner", ax=ax)

plot_qini_curve(y_true=real_y, uplift=t_learner_ite, treatment=real_w, perfect=False, random=False, name="T-learner", ax=ax)

plot_qini_curve(y_true=real_y, uplift=causal_forest_ite, treatment=real_w, perfect=False, name="Causal Forest", ax=ax)

In [ ]:
print("Qini AUC score:")
print(f"S-learner: {qini_auc_score(y_true=real_y, uplift=s_learner_ite, treatment=real_w)}")
print(f"T-learner: {qini_auc_score(y_true=real_y, uplift=t_learner_ite, treatment=real_w)}")
print(f"Causal forest: {qini_auc_score(y_true=real_y, uplift=causal_forest_ite, treatment=real_w)}")